Exercicio 2 PCS5024, Mais algoritmos para a base adult kaggle

In [ ]:
import FuncoesAuxiliares as fa
import FuncoesDeMachineLearning as fml
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [ ]:
datasetPath = f"Kaggle\\adult.csv"
datasetDataFrame = fa.getDataframeInSpecificFormat(datasetPath)

Descricao das colunas discretas da base 

In [ ]:
describedDataFrame = pd.DataFrame(columns = None)
for column in datasetDataFrame.columns:
    if (datasetDataFrame[column].dtype == 'object'):
        describedDataFrame[column] = datasetDataFrame[column].describe(include=['category'])
describedDataFrame

In [ ]:
datasetDataFrame['workclass'].value_counts()

Transforma classes binarias em variaveis binarias

In [ ]:
outputName = 'income'
datasetDataFrameReducedColumns = datasetDataFrame.copy() 
datasetDataFrameReducedColumns[outputName] =  datasetDataFrame["income"].map({"<=50K":0, ">50K":1})
datasetDataFrameReducedColumns['sex'] =  datasetDataFrame["sex"].map({"Male":1, "Female":0})
# Coluna casado ou não
datasetDataFrameReducedColumns["marital.status"] = datasetDataFrameReducedColumns["marital.status"].replace(['Never-married','Divorced','Separated','Widowed'], 'Single')
datasetDataFrameReducedColumns["marital.status"] = datasetDataFrameReducedColumns["marital.status"].replace(['Married-civ-spouse','Married-spouse-absent','Married-AF-spouse'],'Married')
datasetDataFrameReducedColumns["marital.status"] = datasetDataFrameReducedColumns["marital.status"].map({"Married":1, "Single":0})
# dropa a coluna de education como int
datasetDataFrameReducedColumns.drop(['education.num'],axis = 1,inplace=True)

Porcentagem de nulos em cada uma das colunas

In [ ]:
dicOfMissingValues = {}
sizeOfData = len(datasetDataFrameReducedColumns.index)
for column in datasetDataFrameReducedColumns.columns:
    dicOfMissingValues[column] = list(datasetDataFrame[column]).count("?")*100/sizeOfData
print("Porcentagem de '?' em cada coluna ")
for key,val in  dicOfMissingValues.items(): 
    print('{key} - {val:.2f}%'.format(key = key, val = val))

Remove as linhas que possuem valores nulos

In [ ]:
datasetDataFrameWithoutColumnsWithMissingValues = fa.removeLinesWithMissingValues(datasetDataFrameReducedColumns)
print(f"Numero total de linhas = {len(datasetDataFrameReducedColumns.index)}")
print(f"Numero total de linhas que não possuem valores nulos = {len(datasetDataFrameWithoutColumnsWithMissingValues.index)}")
print("Porcentagem de linhas perdidas = ")
print(f"{(1 - len(datasetDataFrameWithoutColumnsWithMissingValues.index)/len(datasetDataFrameReducedColumns.index))*100:.2f}%")

In [ ]:
describedDataFrame = pd.DataFrame(columns = None)
for column in datasetDataFrameWithoutColumnsWithMissingValues.columns:
    if (datasetDataFrameWithoutColumnsWithMissingValues[column].dtype != 'object'):
        describedDataFrame[column] = datasetDataFrameWithoutColumnsWithMissingValues[column].describe(include=['category'])
describedDataFrame

In [ ]:
datasetDataFrameWithoutColumnsWithMissingValues['relationship'].value_counts()

Mostra imagem de correlação

In [ ]:
dummiesDataFrame = pd.get_dummies(datasetDataFrameWithoutColumnsWithMissingValues)
cor = dummiesDataFrame.corr()
cor_target = abs(cor[outputName]).sort_values(ascending=False)[1:]
cor_target

In [ ]:
numberOfRelevantVariables = 5
columnsNamesToBeRemoved = list(cor_target.keys()[numberOfRelevantVariables:])
correlatedDataFrame = dummiesDataFrame.drop(columns = columnsNamesToBeRemoved)
plt.figure(figsize=(50,40))
cor = correlatedDataFrame.corr()
sns.set(font_scale=4)
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds, annot_kws={"size": 50})

Create training validaditon and test sets

In [57]:
numberOfRelevantVariables = 20
X_train,X_crossVal, X_test,y_train,y_crossVal, y_test = fa.prepareDatasetforTraining(datasetDataFrameWithoutColumnsWithMissingValues,outputName,numberOfRelevantVariables)

Cross Validating algorithms

In [ ]:
#KNN algorithm
knnPerformanceDataFrame = fml.applyKNNWithDifferentHyperparameters(X_train, X_crossVal, y_train, y_crossVal,numberOfRelevantVariables)
knnPerformanceDataFrame.to_excel('knnPeformance.xlsx')
knnPerformanceDataFrame

In [ ]:
#Naieve bayes
naiveBayesPerfomanceDataFrame = fml.applyNaiveBayesWithDifferentHyperparameters(X_train, X_crossVal, y_train, y_crossVal,numberOfRelevantVariables)
naiveBayesPerfomanceDataFrame.to_excel('NBPeformance.xlsx')
naiveBayesPerfomanceDataFrame

In [ ]:
#Logistic Regression
logisticRegressionPerformanceDataFrame = fml.applyLogisticRegressionDifferentHyperparameters(X_train, X_crossVal, y_train, y_crossVal,numberOfRelevantVariables)
logisticRegressionPerformanceDataFrame.to_excel('LRPeformance.xlsx')
logisticRegressionPerformanceDataFrame

In [ ]:
#Decision Tree
decisionTreePerformanceDataFrame = fml.applyDecisionTreeDifferentHyperparameters(X_train, X_crossVal, y_train, y_crossVal,numberOfRelevantVariables)
decisionTreePerformanceDataFrame.to_excel('DTPeformance.xlsx')
decisionTreePerformanceDataFrame

In [ ]:
#Random forest
randomForestPerformanceDataFrame = fml.applyRandomForestDifferentHyperparameters(X_train, X_crossVal, y_train, y_crossVal,numberOfRelevantVariables)
randomForestPerformanceDataFrame.to_excel('RFPeformance.xlsx')
randomForestPerformanceDataFrame

Treinamento final:

In [58]:
X_train.shape, X_crossVal.shape,X_new_train.shape,X_test.shape

((18096, 20), (6033, 20), (24129, 20), (6033, 20))

In [59]:
X_new_train = np.concatenate((X_train, X_crossVal), axis=0)
y_new_train = np.concatenate((y_train, y_crossVal), axis=0)

In [60]:
numberOfRelevantVariables = 20
randomForestPerformanceDataFrame = fml.applyRandomForestDifferentHyperparameters(X_new_train, X_test, y_new_train, y_test,numberOfRelevantVariables)
randomForestPerformanceDataFrame

,max_depth,n_estimator,number of relevant variables,accuracy,precision,recall,f1-score
0,10,200,20,0.852810,0.776673,0.572667,0.659248
1,35,200,20,0.835571,0.702229,0.588000,0.640058
2,60,200,20,0.835571,0.702552,0.587333,0.639797
3,85,200,20,0.835571,0.702552,0.587333,0.639797
4,110,200,20,0.835571,0.702552,0.587333,0.639797
5,10,650,20,0.851318,0.778908,0.561333,0.652460
6,35,650,20,0.836400,0.708367,0.581333,0.638594
7,60,650,20,0.836400,0.707357,0.583333,0.639386
8,85,650,20,0.836400,0.707357,0.583333,0.639386
9,110,650,20,0.836400,0.707357,0.583333,0.639386


In [61]:
randomForestPerformanceDataFrame.sort_values(by='f1-score', ascending=False)

,max_depth,n_estimator,number of relevant variables,accuracy,precision,recall,f1-score
0,10,200,20,0.852810,0.776673,0.572667,0.659248
5,10,650,20,0.851318,0.778908,0.561333,0.652460
20,10,2000,20,0.851649,0.783505,0.557333,0.651344
10,10,1100,20,0.851152,0.781308,0.557333,0.650584
15,10,1550,20,0.851152,0.781835,0.556667,0.650312
16,35,1550,20,0.838389,0.714636,0.582667,0.641939
14,110,1100,20,0.837726,0.712653,0.582000,0.640734
13,85,1100,20,0.837726,0.712653,0.582000,0.640734
12,60,1100,20,0.837726,0.712653,0.582000,0.640734
1,35,200,20,0.835571,0.702229,0.588000,0.640058
